In [1]:
import pandas as pd
from preprocessing import preprocessor

In [2]:
data = pd.read_csv("InputFile.csv")

In [3]:
data.shape

(75083, 11)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75083 entries, 0 to 75082
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   DATE              75083 non-null  object 
 1   VISIBILITY        75083 non-null  float64
 2   DRYBULBTEMPF      75083 non-null  int64  
 3   WETBULBTEMPF      75083 non-null  int64  
 4   DewPointTempF     75083 non-null  int64  
 5   RelativeHumidity  75083 non-null  int64  
 6   WindSpeed         75083 non-null  int64  
 7   WindDirection     75083 non-null  int64  
 8   StationPressure   75083 non-null  float64
 9   SeaLevelPressure  75083 non-null  float64
 10  Precip            75083 non-null  float64
dtypes: float64(4), int64(6), object(1)
memory usage: 6.3+ MB


In [5]:
data.head(5)

,DATE,VISIBILITY,DRYBULBTEMPF,WETBULBTEMPF,DewPointTempF,RelativeHumidity,WindSpeed,WindDirection,StationPressure,SeaLevelPressure,Precip
0,01-01-2010 00:51,6.0,33,32,31,92,0,0,29.97,29.99,0.01
1,01-01-2010 01:51,6.0,33,33,32,96,0,0,29.97,29.99,0.02
2,01-01-2010 02:51,5.0,33,33,32,96,0,0,29.97,29.99,0.02
3,01-01-2010 03:51,5.0,33,33,32,96,0,0,29.95,29.97,0.02
4,01-01-2010 04:51,5.0,33,32,31,92,0,0,29.93,29.96,0.02


In [11]:
data = preprocessor.remove_columns(data,['Precip'])

In [12]:
data.head(5)

,DATE,VISIBILITY,DRYBULBTEMPF,WETBULBTEMPF,DewPointTempF,RelativeHumidity,WindSpeed,WindDirection,StationPressure,SeaLevelPressure
0,01-01-2010 00:51,6.0,33,32,31,92,0,0,29.97,29.99
1,01-01-2010 01:51,6.0,33,33,32,96,0,0,29.97,29.99
2,01-01-2010 02:51,5.0,33,33,32,96,0,0,29.97,29.99
3,01-01-2010 03:51,5.0,33,33,32,96,0,0,29.95,29.97
4,01-01-2010 04:51,5.0,33,32,31,92,0,0,29.93,29.96


In [15]:
data = preprocessor.dropUnnecessaryCol(data,['DATE','WETBULBTEMPF','DewPointTempF','StationPressure'])

In [16]:
data.head(5)

,VISIBILITY,DRYBULBTEMPF,RelativeHumidity,WindSpeed,WindDirection,SeaLevelPressure
0,6.0,33,92,0,0,29.99
1,6.0,33,96,0,0,29.99
2,5.0,33,96,0,0,29.99
3,5.0,33,96,0,0,29.97
4,5.0,33,92,0,0,29.96


In [17]:
X, Y = preprocessor.separate_label_feature(data=data, label_column_name='VISIBILITY')

In [18]:
X.shape,Y.shape

((75083, 5), (75083,))

## EDA

In [ ]:
## Another notebook

## Validation of Batch Files

In [1]:
import sqlite3
from datetime import datetime
from os import listdir
import os
import re
import json
import shutil
import pandas as pd

In [2]:
class Raw_Data_validation:
    
    def __init__(self,path):
        self.Batch_Directory = path
        self.schema_path = 'schema.json'
    def valuesFromSchema(self):
        # it will extract all the information from given schema
        f = open(self.schema_path,'r')
        dict1 = json.load(f)  # it will return json object containing data in key-value pairs
        f.close()
        #print(type(dict1))
        pattern = dict1['SampleFileName']
        LengthOfDateStampInFile = dict1['LengthOfDateStampInFile']
        LengthOfTimeStampInFile = dict1['LengthOfTimeStampInFile']
        column_names = dict1['ColName']
        NumberofColumns = dict1['NumberofColumns']
        
        return LengthOfDateStampInFile, LengthOfTimeStampInFile, column_names, NumberofColumns
    
    def FileNameRegex(self):
        #regular exp for the filename from training batch files
        regex = "['visibility']+['\_'']+[\d_]+[\d]+\.csv"
        return regex
    def validationFileName(self,regex,LengthOfDateStampInFile,LengthOfTimeStampInFile):
        # file name validation with regex and schema info
        
        onlyfiles = [f for f in listdir(self.Batch_Directory)] # this will give all the files at given path
        destination="Training_files_validated/Good_data"
        destination2="Training_files_validated/Bad_data"
        for filename in onlyfiles:
                if (re.match(regex, filename)):
                    split1 = re.split('.csv', filename)
                    split2 = (re.split('_', split1[0]))
                    if len(split2[1]) == LengthOfDateStampInFile:
                        
                        if len(split2[2]) == LengthOfTimeStampInFile:
                            shutil.copy("Training_Batch_Files/" + filename, "Training_files_validated/Good_data")
                        else:
                            shutil.copy("Training_Batch_Files/" + filename, "Training_files_validated/Bad_data")
                    else:
                        shutil.copy("Training_Batch_Files/" + filename, "Training_files_validated/Bad_data")
                else:
                    shutil.copy("Training_Batch_Files/" + filename, "Training_files_validated/Bad_data")
        
    def validateColumnLength(self,NumberofColumns):
        # even if file name is right ,it may happen that no. of cols are not same
        # so this function will validate that.
        
        for file in listdir('Training_files_validated/Good_data/'):
            csv = pd.read_csv("Training_files_validated/Good_data/" + file)
            if csv.shape[1] == NumberofColumns:  #shape gives(rows,columns) so index 1
                pass
            else:
                shutil.move("Training_files_validated/Good_data/" + file, "Training_files_validated/Bad_data")
            

## database operations

In [3]:
import csv

In [4]:
class dBOperation:           #This class is used for handling all the SQL operations.
    
    def __init__(self):
        self.badFilePath = "Training_files_validated/Bad_data"
        self.goodFilePath = "Training_files_validated/Good_data"
    def dataBaseConnection(self,DatabaseName):
        # This method creates the database with the given name and 
        # if Database already exists then opens the connection to the DB.
        
        conn = sqlite3.connect(DatabaseName+'.db')
        return conn
    
    def createTableDb(self,DatabaseName,column_names):
        
    #This method creates a table in the given database which will be used to insert the Good data
        conn = self.dataBaseConnection(DatabaseName)
        for key in column_names.keys():
            type = column_names[key]
            try:
                conn.execute('ALTER TABLE Good_Raw_Data ADD COLUMN "{column_name}" {dataType}'.format(column_name=key,dataType=type))
            except:
                
                conn.execute('CREATE TABLE  Good_Raw_Data ({column_name} {dataType})'.format(column_name=key, dataType=type))
        conn.close()
        
    def insertIntoTableGoodData(self,Database):
        conn = self.dataBaseConnection(Database)
        goodFilePath= self.goodFilePath
        badFilePath = self.badFilePath
        onlyfiles = [f for f in listdir(goodFilePath)]
        
        for file in onlyfiles:
            
            with open(goodFilePath+'/'+file, "r") as f:
                next(f)
                reader = csv.reader(f, delimiter="\n")
                for line in enumerate(reader):
                    for list_ in (line[1]):
                        conn.execute('INSERT INTO Good_Raw_Data values ({values})'.format(values=(list_)))
                        conn.commit()
                

        conn.close()
        
        
            
    def selectingDatafromtableintocsv(self,Database):
        #This method exports the data from Good_Raw_Data table as a CSV file. at a given location.
        
        
        self.fileName = 'InputFile.csv'
        conn = self.dataBaseConnection(Database)
        sqlSelect = "SELECT *  FROM Good_Raw_Data"
        cursor = conn.cursor()

        cursor.execute(sqlSelect)
        results = cursor.fetchall()
        
        # Get the headers of the csv file
        headers = [i[0] for i in cursor.description]  #description property will return a list of tuples describing the columns
        # 0th index is always a col name in description 
        with open( self.fileName, 'w', newline='') as csvFile:
            csvFile = csv.writer(csvFile,delimiter=',',lineterminator='\n')
        
            csvFile.writerow(headers)   # for single row at a time--to write the field names or col names
            csvFile.writerows(results)  # for multiple rows at a time
        
        

In [5]:
class train_validation:
    def __init__(self,path):
        self.raw_data = Raw_Data_validation(path)
        self.dBOperation = dBOperation()
        
    def train_validation(self):
        
            # extracting values from prediction schema
            LengthOfDateStampInFile, LengthOfTimeStampInFile, column_names, noofcolumns = self.raw_data.valuesFromSchema()
            # getting the regex defined to validate filename
            regex = self.raw_data.FileNameRegex()
            # validating filename of prediction files
            self.raw_data.validationFileName(regex, LengthOfDateStampInFile, LengthOfTimeStampInFile)
            # validating column length in the file
            self.raw_data.validateColumnLength(noofcolumns)
            
           
            # create database with given name, if present open the connection! Create table with columns given in schema
            self.dBOperation.createTableDb('Training', column_names)
           
            # insert csv files in the table
            self.dBOperation.insertIntoTableGoodData('Training')
            

            # export data in table to csvfile
            self.dBOperation.selectingDatafromtableintocsv('Training')
            
